In [8]:
! pip install transformers
! pip install torch
! pip install datasets
! pip install sentence_transformers

  Obtaining dependency information for sentence_transformers from https://files.pythonhosted.org/packages/75/0c/0bbbf03748c3c7c69f41f016b14cbee946cbd8880d0fb91a05c6f7b7a176/sentence_transformers-3.1.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.6 MB/s eta 0:00:00a 0:00:01


In [10]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import torch

In [4]:
dataset = load_dataset("multi_news", split="test")

In [6]:
df = dataset.to_pandas().sample(2000, random_state=42)
df

,document,summary
4830,Tweet with a location \n \n You can add locati...,– Denis Finley has taken to Twitter to call Po...
1255,CNN host Piers Morgan just called to discuss h...,– CNN's Piers Morgan thinks gun-rights propone...
80,White House communications director Anthony Sc...,– New White House communications director Anth...
3044,CLOSE Scientists say they've found archaeologi...,– Scientists say they have the first physical ...
4486,Click image above to view graphic \n \n Althou...,– Scientists are calling it a breakthrough and...
...,...,...
2157,"On Thursday afternoon, President-elect Donald ...","– He who pays the piper calls the tune, and it..."
3615,Donald Trump said Sunday that in the wake of t...,– In the wake of the Orlando shooting one week...
2751,Nashua police believe body found is that of mi...,"– Sad news out of Nashua, NH, after police say..."
622,The public school systems in New York and Los ...,"– Some 640,000 kids in the nation's second-lar..."


In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2")

/Users/steve/miniconda3/envs/llm/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
passage_embeddings = list(model.encode(df["summary"].tolist()))

In [13]:
# query = "Find me some articles about technology and AI"
query = "Natural disasters"

query_embedddings = model.encode(query)

similarities = util.cos_sim(query_embedddings, passage_embeddings)

top_indices = torch.topk(similarities.flatten(), k=3).indices

top_relevant_passages = [df.iloc[x.item()]["summary"][:200]+"..." for x in top_indices]
top_relevant_passages

['– The tsunami that killed hundreds, possibly thousands of people after an earthquake in Indonesia on Friday was much bigger and more devastating than would normally be expected after that kind of quak...',
 '– A sad milestone out of Japan: Two weeks after the quake struck, its official death toll has broken the 10,000 mark—and that number is still on the rise, with more than 17,400 missing. Police estimat...',
 '– When you live near a major dam, the last thing you want to hear is that the integrity of it has been "compromised" by landslides. But that\'s exactly what residents living below North Carolina\'s Lake...']